# 02-01 : Initial Test Suite

Use the the course work [MD2 test suite](https://learn.london.ac.uk/pluginfile.php/254834/mod_resource/content/1/test%20suite.pdf) a pre-made library to build the unit tests.

## MD2 test suite

```
MD2 ("") = 8350e5a3e24c153df2275c9f80692773
MD2 ("a") = 32ec01ec4a6dac72c0ab96fb34c0b5d1
MD2 ("abc") = da853b0d3f88d99b30283a69e6ded6bb
MD2 ("message digest") = ab4f496bfb2a530b219ff33031fe06b0
MD2 ("abcdefghijklmnopqrstuvwxyz") = 4e8ddff3650292ab5a4108c3aa47940b
MD2 ("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789") =
da33def2a42df13975352846c30338cd
MD2 ("123456789012345678901234567890123456789012345678901234567890123456
78901234567890") = d5976f79d83d3a0dc9806c3c66f3efd8
```

In [1]:
import unittest

## PyCryptodome

This library is simply used as a reference for the MD2 hash function verification.

In [2]:
import Crypto.Hash.MD2

In [3]:
def library_md2(text):
    """
    Generate a MD2 hash of the text using the PyCryptodome library.

    Parameters
    ----------
    text : str
        The text to be hashed.
    """
    hashObject = Crypto.Hash.MD2.new()
    hashObject.update(text.encode('utf-8'))
    digest = hashObject.hexdigest()
    return digest

# test the function
MD2 = library_md2
assert MD2 ("") == '8350e5a3e24c153df2275c9f80692773'

## Implement Unit Tests

In [4]:
class TestMD2(unittest.TestCase):
    def test_empty(self):
        """
        Test the MD2 hash of an empty string.
        """
        self.assertEqual(MD2(""), '8350e5a3e24c153df2275c9f80692773')

    def test_md2(self):
        self.assertEqual(MD2(""), '8350e5a3e24c153df2275c9f80692773')
        self.assertEqual(MD2("a"), '32ec01ec4a6dac72c0ab96fb34c0b5d1')
        self.assertEqual(MD2("abc"), 'da853b0d3f88d99b30283a69e6ded6bb')
        self.assertEqual(MD2("message digest"), 'ab4f496bfb2a530b219ff33031fe06b0')
        self.assertEqual(MD2("abcdefghijklmnopqrstuvwxyz"), '4e8ddff3650292ab5a4108c3aa47940b')
        self.assertEqual(MD2("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789"), 'da33def2a42df13975352846c30338cd')
        self.assertEqual(MD2("12345678901234567890123456789012345678901234567890123456789012345678901234567890"), 'd5976f79d83d3a0dc9806c3c66f3efd8')

# execute the unit tests
MD2 = library_md2
unittest.main(argv=['ignored', '-v'], exit=False)

test_empty (__main__.TestMD2)
Test the MD2 hash of an empty string. ... ok
test_md2 (__main__.TestMD2) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


## MD2 Implementation

In [106]:
import binascii

class MD2Hash(object):
    # the substitution values used to compute the checksum
    S = [
        41, 46, 67, 201, 162, 216, 124, 1, 61, 54, 84, 161, 236, 240, 6, 19,
        98, 167, 5, 243, 192, 199, 115, 140, 152, 147, 43, 217, 188, 76, 130, 202,
        30, 155, 87, 60, 253, 212, 224, 22, 103, 66, 111, 24, 138, 23, 229, 18,
        190, 78, 196, 214, 218, 158, 222, 73, 160, 251, 245, 142, 187, 47, 238, 122,
        169, 104, 121, 145, 21, 178, 7, 63, 148, 194, 16, 137, 11, 34, 95, 33,
        128, 127, 93, 154, 90, 144, 50, 39, 53, 62, 204, 231, 191, 247, 151, 3,
        255, 25, 48, 179, 72, 165, 181, 209, 215, 94, 146, 42, 172, 86, 170, 198,
        79, 184, 56, 210, 150, 164, 125, 182, 118, 252, 107, 226, 156, 116, 4, 241,
        69, 157, 112, 89, 100, 113, 135, 32, 134, 91, 207, 101, 230, 45, 168, 2,
        27, 96, 37, 173, 174, 176, 185, 246, 28, 70, 97, 105, 52, 64, 126, 15,
        85, 71, 163, 35, 221, 81, 175, 58, 195, 92, 249, 206, 186, 197, 234, 38,
        44, 83, 13, 110, 133, 40, 132, 9, 211, 223, 205, 244, 65, 129, 77, 82,
        106, 220, 55, 200, 108, 193, 171, 250, 36, 225, 123, 8, 12, 189, 177, 74,
        120, 136, 149, 139, 227, 99, 232, 109, 233, 203, 213, 254, 59, 0, 29, 57,
        242, 239, 183, 14, 102, 88, 208, 228, 166, 119, 114, 248, 235, 117, 75, 10,
        49, 68, 80, 180, 143, 237, 31, 26, 219, 153, 141, 51, 159, 17, 131, 20]

    def __init__(self):
        """
        Initialize the hash object.
        """

    def append_padding(self, message_bytes:bytearray) -> bytearray:
        """
        The message is "padded" (extended) so that its length (in bytes) is
        congruent to 0, modulo 16. That is, the message is extended so that
        it is a multiple of 16 bytes long. Padding is always performed, even
        if the length of the message is already congruent to 0, modulo 16.

        Padding is performed as follows: "i" bytes of value "i" are appended Expand
        to the message so that the length in bytes of the padded message
        becomes congruent to 0, modulo 16. At least one byte and at most 
        16 bytes are appended.

        At this point the resulting message (after padding with bytes) has a
        length that is an exact multiple of 16 bytes. Let M[0 ... N-1] denote
        the bytes of the resulting message, where N is a multiple of 16.
        """
        padding_length = 16 - (len(message_bytes) % 16)
        return message_bytes + bytearray([padding_length] * padding_length)

    def append_checksum(self, M:bytearray) -> bytearray:
        """
        A 16-byte checksum of the message is appended to the result of the
        previous step.

        This step uses a 256-byte "random" permutation constructed from the
        digits of pi. Let S[i] denote the i-th element of this table.        
        """
        # create an empty checksum
        C = bytearray([0] * 16)

        L = 0

        # process each 16-byte block
        for i in range(0, len(M) // 16):
            # compute the checksum for block i
            for j in range(0, 16):
                c = M[i * 16 + j]
                C[j] = C[j] ^ self.S[c ^ L]
                L = C[j]

        return M + C

    def process_message(self, message_bytes:bytearray) -> bytearray:
        """
        The message is processed as follows:

        1. The message is padded to a length that is an exact multiple of
           16 bytes.
        2. The message is extended with a 16-byte checksum.
        3. The message digest is computed.
        """
        # initialize the md buffer
        X = bytearray([0] * 48)

        # get the message with padding and the checksum appended
        M = self.append_checksum(
            self.append_padding(message_bytes)
        )

        # process each 16-byte block
        for i in range(0, len(M) // 16):
            # copy block i into X
            for j in range(0, 16):
                X[16 + j] = M[i * 16 + j]
                X[32 + j] = X[16 + j] ^ X[j]

            t = 0

            # do 18 rounds
            for j in range(0, 18):
                # round j
                for k in range(0, 48):
                    t = X[k] ^ self.S[t]
                    X[k] = t

                t = (t + j) % 256

        return X

    def MD2(self, message:str) -> str:
        """
        Compute the message digest for a given message.
        """
        md_buffer = self.process_message(
            bytearray(message.encode('utf-8'))
        )

        return binascii.hexlify(md_buffer[:16]).decode('utf-8')


class TestMD2Hash(unittest.TestCase):
    _test_messages = [
        '',
        'a',
        'abc',
        'message digest',
        '1234567890abcdef',
        'abcdefghijklmnopqrstuvwxyz',
        'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789',
        '12345678901234567890123456789012345678901234567890123456789012345678901234567890'
    ]

    def test_s(self):
        """
        Ensure that the value for S was loaded correctly.
        """
        self.assertEqual(MD2Hash.S[0], 41)
        self.assertEqual(MD2Hash.S[255], 20)

    def test_padding(self):
        """
        test the padding function.
        """
        hashObject = MD2Hash()

        for message in self._test_messages:
            message_bytes = bytearray(message, 'utf-8')
            padded_bytes = hashObject.append_padding(message_bytes)
        
            # padding must always be performed
            self.assertGreater(len(padded_bytes), len(message_bytes))
            
            # the padded message must be a multiple of 16 bytes
            self.assertEqual(len(padded_bytes) % 16, 0)

            # the padding character must be the length of the padding
            padding_length = 16 - (len(message_bytes) % 16)
            self.assertEqual(padded_bytes[-1], padding_length)

    def test_checksum(self):
        """
        test the checksum function.
        """
        hashObject = MD2Hash()

        for message in self._test_messages:
            message_bytes = bytearray(message, 'utf-8')
            padded_bytes = hashObject.append_padding(message_bytes)
            checksum_bytes =hashObject.append_checksum(padded_bytes)

            # the checksum must be 16 bytes long
            self.assertEqual(len(checksum_bytes) - len(padded_bytes), 16)

    def test_process_message(self):
        """
        Test computing the message digest.
        """
        hashObject = MD2Hash()

        for message in self._test_messages:
            message_bytes = bytearray(message, 'utf-8')
            message_buffer_bytes = hashObject.process_message(message_bytes)

            # the message digest must be 48 bytes long
            self.assertEqual(len(message_buffer_bytes), 48)

    def test_md2(self):
        hashObject = MD2Hash()

        self.assertEqual(hashObject.MD2(""), '8350e5a3e24c153df2275c9f80692773')
        self.assertEqual(hashObject.MD2("a"), '32ec01ec4a6dac72c0ab96fb34c0b5d1')
        self.assertEqual(hashObject.MD2("abc"), 'da853b0d3f88d99b30283a69e6ded6bb')
        self.assertEqual(hashObject.MD2("message digest"), 'ab4f496bfb2a530b219ff33031fe06b0')
        self.assertEqual(hashObject.MD2("abcdefghijklmnopqrstuvwxyz"), '4e8ddff3650292ab5a4108c3aa47940b')
        self.assertEqual(hashObject.MD2("ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789"), 'da33def2a42df13975352846c30338cd')
        self.assertEqual(hashObject.MD2("12345678901234567890123456789012345678901234567890123456789012345678901234567890"), 'd5976f79d83d3a0dc9806c3c66f3efd8')

unittest.main(argv=['ignored', '-v'], exit=False)

test_empty (__main__.TestMD2)
Test the MD2 hash of an empty string. ... ok
test_md2 (__main__.TestMD2) ... ok
test_checksum (__main__.TestMD2Hash)
test the checksum function. ... ok
test_md2 (__main__.TestMD2Hash) ... ok
test_padding (__main__.TestMD2Hash)
test the padding function. ... ok
test_process_message (__main__.TestMD2Hash)
Test computing the message digest. ... ok
test_s (__main__.TestMD2Hash)
Ensure that the value for S was loaded correctly. ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.022s

OK


In [107]:
def custom_md2(message:str) -> str:
    hashObject = MD2Hash()
    return hashObject.MD2(message)

MD2 = custom_md2
unittest.main(argv=['ignored', '-v'], exit=False)

test_empty (__main__.TestMD2)
Test the MD2 hash of an empty string. ... ok
test_md2 (__main__.TestMD2) ... ok
test_checksum (__main__.TestMD2Hash)
test the checksum function. ... ok
test_md2 (__main__.TestMD2Hash) ... ok
test_padding (__main__.TestMD2Hash)
test the padding function. ... ok
test_process_message (__main__.TestMD2Hash)
Test computing the message digest. ... ok
test_s (__main__.TestMD2Hash)
Ensure that the value for S was loaded correctly. ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.021s

OK
